In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import nltk
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package stopwords to C:\Users\KHALED
[nltk_data]     SAYED\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\KHALED
[nltk_data]     SAYED\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\KHALED
[nltk_data]     SAYED\AppData\Roaming\nltk_data...


True

In [2]:
df_amazon = pd.read_csv (r'E:\NLP_200\Code Section 05\amazon_alexa.tsv', sep="\t")

In [3]:
df_amazon.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [4]:
df_amazon.shape

(3150, 5)

In [5]:
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   rating            3150 non-null   int64 
 1   date              3150 non-null   object
 2   variation         3150 non-null   object
 3   verified_reviews  3149 non-null   object
 4   feedback          3150 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 123.2+ KB


In [6]:
df_amazon.variation.value_counts()

variation
Black  Dot                      516
Charcoal Fabric                 430
Configuration: Fire TV Stick    350
Black  Plus                     270
Black  Show                     265
Black                           261
Black  Spot                     241
White  Dot                      184
Heather Gray Fabric             157
White  Spot                     109
White                            91
Sandstone Fabric                 90
White  Show                      85
White  Plus                      78
Oak Finish                       14
Walnut Finish                     9
Name: count, dtype: int64

In [11]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(df_amazon['verified_reviews'])):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(df_amazon['verified_reviews'][sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    #document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    #document = document.split()
    document = word_tokenize(document)
    
    
    stop_words = set(stopwords.words('english'))
    document = [w for w in document if not w in stop_words]
    
        
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [12]:
pd.DataFrame(documents)

,0
0,love echo
1,loved
2,sometimes playing game answer question correct...
3,lot fun thing 4 yr old learns dinosaur control...
4,music
...,...
3145,perfect kid adult everyone
3146,listening music searching location checking ti...
3147,love thing running entire home tv light thermo...
3148,complaint sound quality great mostly use comma...


In [15]:
from sklearn.model_selection import train_test_split

x= documents
y=df_amazon['feedback']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

#print('Training Data Shape:' , X_train.shape)
#print('Training Data Shape:' , X_test.shape)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect= CountVectorizer()

X_train_counts= count_vect.fit_transform(X_train)
X_train_counts.shape 

(2110, 3095)

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2110, 3095)

In [18]:
pd.DataFrame(X_train_tfidf)[0]

0         (0, 3055)\t0.36014260152406846\n  (0, 1656)\...
1         (0, 3007)\t0.21940950499928316\n  (0, 2934)\...
2         (0, 2929)\t0.1019125778027732\n  (0, 2779)\t...
3         (0, 2796)\t0.34905904616190875\n  (0, 2426)\...
4         (0, 2634)\t0.2599828105047225\n  (0, 2483)\t...
                              ...                        
2105      (0, 2929)\t0.5349303982682322\n  (0, 2739)\t...
2106      (0, 2595)\t0.8061743733154008\n  (0, 1656)\t...
2107      (0, 1261)\t0.6513284140060848\n  (0, 1138)\t...
2108      (0, 2915)\t0.23412746358022596\n  (0, 2470)\...
2109      (0, 2425)\t0.7172496114261561\n  (0, 899)\t0...
Name: 0, Length: 2110, dtype: object

In [19]:
from sklearn.svm import LinearSVC 
clf =LinearSVC()
clf.fit(X_train_tfidf,y_train)

C:\Users\KHALED SAYED\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC()

In [21]:
from sklearn.ensemble import RandomForestClassifier

classifierRF = RandomForestClassifier(n_estimators=1000, random_state=0)
classifierRF.fit(X_train_tfidf, y_train)

RandomForestClassifier(n_estimators=1000, random_state=0)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(2110, 3095)

In [23]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),])

text_RF = Pipeline([('tfidf', TfidfVectorizer()),
                     ('classifierRF', RandomForestClassifier()),])


text_clf.fit(X_train, y_train)  
text_RF.fit(X_train, y_train)


predictions_text_clf = text_clf.predict(X_test)
predictions_text_RF = text_RF.predict(X_test)

C:\Users\KHALED SAYED\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [24]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions_text_clf))
print(classification_report(y_test,predictions_text_clf))
print(accuracy_score(y_test, predictions_text_clf))

[[ 34  65]
 [  6 935]]
              precision    recall  f1-score   support

           0       0.85      0.34      0.49        99
           1       0.94      0.99      0.96       941

    accuracy                           0.93      1040
   macro avg       0.89      0.67      0.73      1040
weighted avg       0.93      0.93      0.92      1040

0.9317307692307693


In [25]:
print(confusion_matrix(y_test,predictions_text_RF))
print(classification_report(y_test,predictions_text_RF))
print(accuracy_score(y_test, predictions_text_RF))

[[ 26  73]
 [  0 941]]
              precision    recall  f1-score   support

           0       1.00      0.26      0.42        99
           1       0.93      1.00      0.96       941

    accuracy                           0.93      1040
   macro avg       0.96      0.63      0.69      1040
weighted avg       0.93      0.93      0.91      1040

0.9298076923076923


In [26]:
text_RF.predict(['sometimes playing game answer question correct'])[0]

1